## Import packages and app key

In [ ]:
import refinitiv.dataplatform as rdp
import refinitiv.dataplatform.legacy as ek

import pandas as pd
import plotly.express as px

rdp.open_desktop_session('DEFAULT_CODE_BOOK_APP_KEY')

#Displays all the columns and rows - useful in the dev process
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

## <u> Gather data </u>

## Search for RICs, get it to a list

#### Example: Search for Brazil global sovereign bonds
            in USD with a duration greater than 1

In [ ]:
df = rdp.search(
    view=rdp.SearchViews.FixedIncomeInstruments,
    filter = "IndustrySectorDescription eq 'Sovereign' and \
              IssuerCountryName eq 'Brazil' and \
              OptionAdjustedDuration gt 1 and \
              Currency eq 'USD'",
    select ="RIC, ISIN, MaturityDate, CouponRate ,Currency,IndustrySector, FaceOutstandingUSD, DocumentTitle, IssuerCountryName, OptionAdjustedDuration",
    top=100
    )
df


### Convert RICs from search results to 'list' type format
#### get_data wants a list!

In [ ]:
rics = df['RIC']
ricList = df['RIC'].tolist()   # Get data expects a list, not a series

display(ricList)
display(type(rics))
display(type(ricList))


### Set the fields

In [ ]:
#Bringing in my fields - the Data Item Browser and Code Creator are your friends!
#I used both the DIB and CODECD

fields = ['CF_DATE',
          'TR.CouponRate',
          'TR.ISIN',
          'TR.IssuerRating',
          'TR.FiDescription',
          'TR.FiFirstCouponDate',
          'TR.FIIssuerName',
          'TR.FiMaturityDate',
          'TR.CA.AmtOutstanding',
          'TR.BENCHMARKSPREAD',
          'TR.ZSPREAD', 
          'TR.MODIFIEDDURATION',   
          'TR.GR.Rating'
          'TR.ASSETSWAPSPREAD',
         ]

### Populate a dataframe with real-time data

In [ ]:
df, err = ek.get_data(ricList, fields)  #pass my rics and fields to get_data function
df

# <u><i> Plot the data... </i></u>

## Simple scatter plot with a <i>single</i> Y variable

In [ ]:
fig = px.scatter(df, x="Modified Duration", y="Z Spread")

fig.show()

### Add a third dimension

In [ ]:
#First, reduce size outstanding to factor in millions
df['Amount Outstanding'] = df['Amount Outstanding']/1000000

In [ ]:
fig = px.scatter(df, x="Modified Duration", y="Z Spread", size='Amount Outstanding',size_max=60)

fig.show()

## Let's plot <i>multiple</i> Y variables - Z Spread vs. Benchmark

### Convert the data to 'long' format for plotly express (Newer versions consume wide-format natively)

In [ ]:
#Original dataframe with a column for both 'Z-Spread' and 'Benchmark Spread'
df.head(1)

In [ ]:
df_long=pd.melt(df,id_vars=['Description','Instrument','Issuer Name','Maturity Date','Coupon Rate','Modified Duration','Amount Outstanding'], 
                value_vars=['Z Spread','Benchmark Spread'],var_name='Spread',value_name='bps') 
df_long

### Plot it

In [ ]:
fig = px.scatter(df_long, x='Modified Duration', y='bps', size='Amount Outstanding', size_max=60)

fig.show()

### Enhance it - hover data, colors, themes

In [ ]:
df_long.info()

In [ ]:
fig = px.scatter(df_long, x='Modified Duration', y='bps',color='Spread',size='Amount Outstanding', size_max=45,   #Change the colors!
                 hover_name='Instrument', 
                 #hover_data=['Desciption']#,'Issuer Name','Coupon Rate','Maturity Date','Modified Duration']# ,'Amount Outstanding'],
                 height=800,
                 template='plotly_dark' #Dark template - matches codebook nicely!
                )
fig.show()

# <u> Another Dimension... </u>

## How would this data look <i>over time</i>?

### Populate a dataframe with time-series data

In [ ]:
#### Set new fields 
ts_fields = ['TR.ZSPREAD','TR.BENCHMARKSPREAD','TR.MODIFIEDDURATION','TR.ZSPREAD.date']  

#Time-series and real-time fields can be different!  Just because you can get a value for something in real-time, doesn't mean you can get it historically.
#The Code Creator (CODECR) app, along with Data Item Browser (DIB) are your friends!
#They help you determine (along with some trial and error), which fields are best suited for your application.

In [ ]:
dfts, err = ek.get_data(ricList, ts_fields,
    parameters = {    #Time-series parameters
        'SDate':'0',
        'EDate':'-365 '  #Last 365 days
    }
)

display(dfts)
display(dfts.info())

In [ ]:
# Note the last bond doesn't have Spread values...  and it shouldn't have - Dated 2019-11-14!
# Check out the RIC in Eikon - you can see the Dated / First / Last Coupon, and it will even give you the field:  TR.FIFirstCouponDate if you want to bring it in later

### Prepare the data for plotting

In [ ]:
dfts.sort_values(by=['Date'],inplace=True)          #Sorts by date for plotting
dfts['Date'] = pd.to_datetime(dfts['Date'])         #Makes it a date field
dfts['Date'] = dfts['Date'].dt.strftime("%Y-%m-%d") #Formats the date
dfts


### Merge the reference data from the first dataframe into my time series dataframe

In [ ]:
#First, let's take a look at the original dataframe, and time-series dataframe:
display(df.head(1))
display(dfts.head(1))

#### Rename one of the "Instrument" field names

In [ ]:
#To merge/join frames, the key names need to be different, so I'll rename the key 'Instrument' to 'RIC' in the original "df" dataframe
df.rename(columns = {'Instrument':'RIC'}, inplace = True)
display(df.head(1))
display(dfts.head(1))

In [ ]:
#Create a new, merged, dataframe 
df_merged = dfts.merge(df[['RIC','Coupon Rate','Description','First Coupon Date','Issuer Name','Maturity Date','Amount Outstanding']], left_on='Instrument', right_on='RIC')
df_merged

### Convert the data to 'long' format

In [ ]:
df_long_ts=pd.melt(df_merged,id_vars=['Date','Instrument','Description','Maturity Date','Coupon Rate','Modified Duration','Amount Outstanding'],
                   value_vars=['Z Spread','Benchmark Spread'],var_name='Spread',value_name='bps') 
df_long_ts.sort_values(by=['Date'],inplace=True) #Resort by date (for plotting reasons)


### Build the scatter plot using 'Date' for the slider

In [ ]:
fig = px.scatter(df_long_ts, x='Modified Duration', y='bps', animation_frame='Date', animation_group='Instrument',
                 size='Amount Outstanding', size_max=45, color="Spread", template='plotly_dark',
                 hover_name='Instrument',
                 #hover_data=['Description','Coupon Rate','Modified Duration','Amount Outstanding','Spread','bps'],
                 #Adjusting the x and y ranges....
                 range_x=[df_long_ts['Modified Duration'].min()/1.1, df_long_ts['Modified Duration'].max()*1.1],  
                 range_y=[df_long_ts['bps'].min()/1.2, df_long_ts['bps'].max()*1.2],
                 height=800
                )

#fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 40 # changes the frame rate to 50 milliseconds
fig.show()

# Curve Spread Example
### Four different spreads, over time - Z Spread, Government Spread, Swap Spread & Asset Swap Spread
### Petroleos Mexicanos USD Global Agency Curve

## <u> Gather data </u>

#### Get RICs from the Chain RIC - you can always put these in manually

In [ ]:
df, err = ek.get_data(
    instruments = ['0#PEMEXUSDGBMK='],
    fields = ['CF_NAME','GV4_TEXT']
)

display(df)

####  Set my RICs

In [ ]:
pem_rics = df['Instrument'][2:17]
pem_rics = [''.join(elemnt) for elemnt in pem_rics]  #Eikon formula wants a list!
display(pem_rics)
display(type(pem_rics))

#### Set my fields

In [ ]:
fields = ['CF_NAME','GV4_TEXT']
          #'TR.BENCHMARKSPREAD','TR.ZSPREAD','TR.SWAPSPREAD','TR.ASSETSWAPSPREAD']

### Get RIC reference data

In [ ]:
pem_df, err = ek.get_data(pem_rics, fields)

pem_df.rename(columns={'GV4_TEXT':'Tenor','CF_NAME':'Name'},inplace=True)  # Rename the columns

pem_df

### Get Time Series data

In [ ]:
ts_fields = ['TR.BENCHMARKSPREAD.date','TR.BENCHMARKSPREAD','TR.ZSPREAD','TR.SWAPSPREAD','TR.ASSETSWAPSPREAD']

In [ ]:
pem_ts, err = ek.get_data(pem_rics, ts_fields,
    parameters = {    #Time-series parameters
        'SDate':'0',
        'EDate':'-365 '  #Last 365 days
    }
)

pem_ts

## Merge the reference data from the first dataframe into my time series dataframe

In [ ]:
pem_df.rename(columns = {'Instrument':'RIC'}, inplace = True)  #Must rename to merge!

In [ ]:
pem_merged = pem_ts.merge(pem_df[['RIC','Name','Tenor']], left_on='Instrument', right_on='RIC')
pem_merged

### Prepare the data for plotting

In [ ]:
#  Create a new column with the Tenor years in Integer form so I can sort them for plotting purposes
pem_merged['TenorYears'] = pem_merged['Tenor'].str.replace(r'\D+', '').astype(int)  #Strips field to just an integer

#  Sort based on TenorYears and Date fields
pem_merged.sort_values(by=['TenorYears','Date'],inplace=True)
pem_merged

#  Human Readable Dates
pem_merged['Date'] = pd.to_datetime(pem_merged['Date'])         #Makes it a date field
pem_merged['Date'] = pem_merged['Date'].dt.strftime("%Y-%m-%d") #Formats the date to something more readable

### Convert to 'Long' format for plotting

In [ ]:
pem_merged_long=pd.melt(pem_merged,id_vars=['Date','Instrument','Name','Tenor','TenorYears'],
                   value_vars=['Z Spread','Benchmark Spread','Swap Spread', 'Asset Swap Spread'],var_name='Spread',value_name='bps') 
pem_merged_long

## Plot the data

In [ ]:
 fig = px.scatter(pem_merged_long, x='Tenor', y='bps', animation_frame='Date', animation_group='Instrument',
                 color="Spread", template='plotly_dark',
                 hover_name='Name',
                 #hover_data=['Spread','bps','Tenor'],
                 range_y=[-50, pem_merged_long['bps'].max()*1.05],  #Adjusting the x and y ranges....
                 height=800
               )

#fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 40 # changes the frame rate in milliseconds
fig.show()